# Inference

In [1]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')


import pandas as pd
import numpy as np
import networkx as nx
import copy
import scipy as sp
import math
import seaborn
import pickle
import warnings
import matplotlib
import re
import multiprocessing

from lib.mobilitysim import MobilitySimulator
from lib.dynamics import DiseaseModel
from lib.inference import * 
from bayes_opt import BayesianOptimization
from lib.parallel import *
from lib.distributions import CovidDistributions
from lib.plot import Plotter
from lib.data import collect_data_from_df
from lib.measures import (
    MeasureList, 
    BetaMultiplierMeasure, 
    BetaMultiplierMeasureByType,
    SocialDistancingForAllMeasure, 
    SocialDistancingByAgeMeasure,
    SocialDistancingForPositiveMeasure, 
    Interval)

from lib.mobilitysim import MobilitySimulator

### Settings

Determine settings for inference. Nothing below should have to be changed signficiantly/at all.

In [2]:
# settings used to generate mobility traces on the fly. These settings are used for inference
# See town-generator.ipynb for an example on how to create
mob_settings = 'lib/tu_settings_20_10.pk'
case_downsample = 10

# parameter bounds
param_bounds = {
    'beta' : {
        'household' : [0.0, 10.0],
        'education' : [0.0, 10.0],
        'office' : [0.0, 10.0],
        'social' : [0.0, 10.0],
        'supermarket' : [0.0, 10.0],
        'transport' : [0.0, 10.0],
    },
    'mu' : [0.0, 1.0]
}

# Based on population size, approx. 300 tests/day in Area of Tübingen (~135 in city of Tübingen)
tests_per_day = 10

# seed for random states and log file
c = 0

## Import Covid19 data

In [3]:
new_cases_ = collect_data_from_df('LK Tübingen', 'new')
resistant_cases_ = collect_data_from_df('LK Tübingen', 'recovered')
fatality_cases_ = collect_data_from_df('LK Tübingen', 'fatality')

Data last updated at:  13.04.2020, 00:00 Uhr
Data last updated at:  13.04.2020, 00:00 Uhr
Data last updated at:  13.04.2020, 00:00 Uhr


Empirical fatality rate per age group from the above data. RKI data defines 6 groups: **0-4y, 5-14y, 15-34y, 35-59y, 60-79y, 80+y**

In [4]:
# fatality rate per age group
num_age_groups = fatality_cases_.shape[1] 
fatality_rates_by_age = (fatality_cases_[-1, :] / \
    (new_cases_[-1, :] +  fatality_cases_[-1, :] + resistant_cases_[-1, :]))

print('Empirical fatality rates per age group:  ', fatality_rates_by_age.tolist())

Empirical fatality rates per age group:   [0.0, 0.0, 0.0, 0.0025252525252525255, 0.005952380952380952, 0.17647058823529413]


Scale down cases based on number of people in simulation

In [5]:
new_cases, resistant_cases, fatality_cases = (
    1/case_downsample * new_cases_, 
    1/case_downsample * resistant_cases_, 
    1/case_downsample * fatality_cases_)
new_cases, resistant_cases, fatality_cases = np.ceil(new_cases), np.ceil(resistant_cases), np.ceil(fatality_cases)

Maximum time fixed by real data

In [6]:
max_time = int(new_cases.shape[0] * 24.0) # maximum time to simulate, in hours

print('Max time T (days):', new_cases.shape[0])
print('Age groups:       ', new_cases.shape[1])
print('Positive at t=0:  ', int(new_cases[0, :].sum()))
print('Positive at t=T:  ', int(new_cases[-1, :].sum()))

Max time T (days): 17
Age groups:        6
Positive at t=0:   3
Positive at t=T:   57


In [7]:
# instantiate correct distributions
distributions = CovidDistributions(fatality_rates_by_age=fatality_rates_by_age)

In [8]:
# set initial seed count (based on infection counts on March 10)
initial_seeds = {
    'expo' : 10,
    'ipre' : 1,
    'isym' : 3,
    'iasy' : 3,
}

In [9]:
# standard quarantine of positive tests and test availablility
measure_list = MeasureList([
    SocialDistancingForPositiveMeasure(
        t_window=Interval(0.0, max_time), p_stay_home=1.0)
])
print('Add here "isolate positive at home"')

Add here "isolate positive at home"


In [10]:
# set testing parameters
testing_params = {
    'testing_t_window'    : [0.0, max_time], # in hours
    'testing_frequency'   : 24.0,     # in hours
    'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
    'tests_per_batch'     : tests_per_day, # assume 300 tests/day in LK Tübingen
    'test_smart_delta'    : 24.0 * 3, # in hours
    'test_smart_duration' : 24.0 * 7, # in hours
    'test_smart_action'   : 'isolate', 
    'test_smart_num_contacts'   : 10, 
    'test_targets'        : 'isym',
    'test_queue_policy'   : 'fifo',
    'smart_tracing'       : None, 
}

## Bayesian optimization 

Load settings as set in header of this notebook and generate example traces to extract information for inference.

In [11]:
'''Advanced loop

*** to write

Multi-task GP paper implemented https://papers.nips.cc/paper/3189-multi-task-gaussian-process-prediction.pdf
(for correlated output dimension, which is the case for us)

drive home difference between objective and noisy observation

add mu xi beta as optimized params


*** to do

- don't forget to standardize
- account for test lag (make sure start of T positive of model is shifted by `test_lag` of test params in loss)
- make sure the compositionality is properly handled 
    (see botorch paper, do need specific objective functionk `GenericCObjective`)
- make sure heteroskedastic GP is fitted 
    (using provided empirical SEM errors, and if so also mention in write-up)
    
    
-  covariates are normalized to the unit cube and outcomes are standardized (zero mean, unit variance).
'''

"Advanced loop\n\n*** to write\n\nMulti-task GP paper implemented https://papers.nips.cc/paper/3189-multi-task-gaussian-process-prediction.pdf\n(for correlated output dimension, which is the case for us)\n\ndrive home difference between objective and noisy observation\n\nadd mu xi beta as optimized params\n\n\n*** to do\n\n- don't forget to standardize\n- account for test lag (make sure start of T positive of model is shifted by `test_lag` of test params in loss)\n- make sure the compositionality is properly handled \n    (see botorch paper, do need specific objective functionk `GenericCObjective`)\n- make sure heteroskedastic GP is fitted \n    (using provided empirical SEM errors, and if so also mention in write-up)\n    \n    \n-  covariates are normalized to the unit cube and outcomes are standardized (zero mean, unit variance).\n"

In [12]:
import gpytorch, torch, botorch, sobol_seq
from botorch import fit_gpytorch_model
from botorch.models import FixedNoiseGP, ModelListGP, HeteroskedasticSingleTaskGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood, MarginalLogLikelihood
from botorch.acquisition.monte_carlo import MCAcquisitionFunction, qNoisyExpectedImprovement, qSimpleRegret
from botorch.acquisition.objective import MCAcquisitionObjective
from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy
from botorch.acquisition import OneShotAcquisitionFunction
import botorch.utils.transforms as transforms
from botorch.utils.transforms import match_batch_shape, t_batch_mode_transform

# from botorch.acquisition import qKnowledgeGradient
from botorch.sampling.samplers import SobolQMCNormalSampler, IIDNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.optim import optimize_acqf
from botorch.acquisition.objective import GenericMCObjective, ConstrainedMCObjective
from botorch.gen import get_best_candidates, gen_candidates_torch
from botorch.optim import gen_batch_initial_conditions

import time, pprint

import warnings
warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

torch.set_printoptions(precision=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [13]:


def gen_one_shot_kg_initial_conditions(
    acq_function,
    bounds,
    q,
    num_restarts,
    raw_samples,
    options=None,
):
    r"""Generate a batch of smart initializations for qKnowledgeGradient.
    This function generates initial conditions for optimizing one-shot KG using
    the maximizer of the posterior objective. Intutively, the maximizer of the
    fantasized posterior will often be close to a maximizer of the current
    posterior. This function uses that fact to generate the initital conditions
    for the fantasy points. Specifically, a fraction of `1 - frac_random` (see
    options) is generated by sampling from the set of maximizers of the
    posterior objective (obtained via random restart optimization) according to
    a softmax transformation of their respective values. This means that this
    initialization strategy internally solves an acquisition function
    maximization problem. The remaining `frac_random` fantasy points as well as
    all `q` candidate points are chosen according to the standard initialization
    strategy in `gen_batch_initial_conditions`.
    Args:
        acq_function: The qKnowledgeGradient instance to be optimized.
        bounds: A `2 x d` tensor of lower and upper bounds for each column of
            task features.
        q: The number of candidates to consider.
        num_restarts: The number of starting points for multistart acquisition
            function optimization.
        raw_samples: The number of raw samples to consider in the initialization
            heuristic.
        options: Options for initial condition generation. These contain all
            settings for the standard heuristic initialization from
            `gen_batch_initial_conditions`. In addition, they contain
            `frac_random` (the fraction of fully random fantasy points),
            `num_inner_restarts` and `raw_inner_samples` (the number of random
            restarts and raw samples for solving the posterior objective
            maximization problem, respectively) and `eta` (temperature parameter
            for sampling heuristic from posterior objective maximizers).
    Returns:
        A `num_restarts x q' x d` tensor that can be used as initial conditions
        for `optimize_acqf()`. Here `q' = q + num_fantasies` is the total number
        of points (candidate points plus fantasy points).
    Example:
        >>> qKG = qKnowledgeGradient(model, num_fantasies=64)
        >>> bounds = torch.tensor([[0., 0.], [1., 1.]])
        >>> Xinit = gen_one_shot_kg_initial_conditions(
        >>>     qKG, bounds, q=3, num_restarts=10, raw_samples=512,
        >>>     options={"frac_random": 0.25},
        >>> )
    """
    options = options or {}
    frac_random: float = options.get("frac_random", 0.1)
    if not 0 < frac_random < 1:
        raise ValueError(
            f"frac_random must take on values in (0,1). Value: {frac_random}"
        )
    q_aug = acq_function.get_augmented_q_batch_size(q=q)

    # TODO: Avoid unnecessary computation by not generating all candidates
    ics = gen_batch_initial_conditions(
        acq_function=acq_function,
        bounds=bounds,
        q=q_aug,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        options=options,
    )

    # compute maximizer of the value function
    value_function = _get_value_function(
        model=acq_function.model,
        objective=acq_function.objective,
        sampler=acq_function.inner_sampler,
    )
#     from .optimize import optimize_acqf

    fantasy_cands, fantasy_vals = optimize_acqf(
        acq_function=value_function,
        bounds=bounds,
        q=1,
        num_restarts=options.get("num_inner_restarts", 20),
        raw_samples=options.get("raw_inner_samples", 1024),
        return_best_only=False,
    )

    # sampling from the optimizers
    n_value = int((1 - frac_random) * (q_aug - q))  # number of non-random ICs
    eta = options.get("eta", 2.0)
    weights = torch.exp(eta * transforms.standardize(fantasy_vals))
    idx = torch.multinomial(weights, num_restarts * n_value, replacement=True)

    # set the respective initial conditions to the sampled optimizers
    ics[..., -n_value:, :] = fantasy_cands[idx, 0].view(num_restarts, n_value, -1)
    return ics

def _split_fantasy_points(X, n_f):
    r"""Split a one-shot optimization input into actual and fantasy points

    Args:
        X: A `batch_shape x (q + n_f) x d`-dim tensor of actual and fantasy
            points

    Returns:
        2-element tuple containing

        - A `batch_shape x q x d`-dim tensor `X_actual` of input candidates.
        - A `n_f x batch_shape x 1 x d`-dim tensor `X_fantasies` of fantasy
            points, where `X_fantasies[i, batch_idx]` is the i-th fantasy point
            associated with the batch indexed by `batch_idx`.
    """
#     print('_split_fantasy_points  ', X.shape) # this is [5, 1, 7] but should actually get [5, (1 + 64), 7]
    
    if n_f > X.size(-2):
        raise ValueError(
            f"n_f ({n_f}) must be less than the q-batch dimension of X ({X.size(-2)})"
        )
    split_sizes = [X.size(-2) - n_f, n_f]
    X_actual, X_fantasies = torch.split(X, split_sizes, dim=-2)
    # X_fantasies is b x num_fantasies x d, needs to be num_fantasies x b x 1 x d
    # for batch mode evaluation with batch shape num_fantasies x b.
    # b x num_fantasies x d --> num_fantasies x b x d
    X_fantasies = X_fantasies.permute(-2, *range(X_fantasies.dim() - 2), -1)
    # num_fantasies x b x 1 x d
    X_fantasies = X_fantasies.unsqueeze(dim=-2)
    return X_actual, X_fantasies


def _get_value_function(
    model,
    objective=None,
    sampler=None,
):
    r"""Construct value function (i.e. inner acquisition function)."""
    if isinstance(objective, MCAcquisitionObjective):
        return qSimpleRegret(model=model, sampler=sampler, objective=objective)
    else:
        return PosteriorMean(model=model, objective=objective)


# class OwnKnowledgeGradient(MCAcquisitionFunction, OneShotAcquisitionFunction):
class qKnowledgeGradient(MCAcquisitionFunction, OneShotAcquisitionFunction):
    r"""
    Copy of qKnowledgeGradient on
    https://github.com/pytorch/botorch/blob/master/botorch/acquisition/knowledge_gradient.py
    
    with a bug fixed that disables using the original qKnowledgeGradient class
    
    =======
    
    Batch Knowledge Gradient using one-shot optimization.

    This computes the batch Knowledge Gradient using fantasies for the outer
    expectation and either the model posterior mean or MC-sampling for the inner
    expectation.

    In addition to the design variables, the input `X` also includes variables
    for the optimal designs for each of the fantasy models. For a fixed number
    of fantasies, all parts of `X` can be optimized in a "one-shot" fashion.
    """

    def __init__(
        self,
        model,
        num_fantasies=64,
        sampler=None,
        objective=None,
        inner_sampler=None,
        X_pending=None,
        current_value=None,
    ):
        r"""q-Knowledge Gradient (one-shot optimization).

        Args:
            model: A fitted model. Must support fantasizing.
            num_fantasies: The number of fantasy points to use. More fantasy
                points result in a better approximation, at the expense of
                memory and wall time. Unused if `sampler` is specified.
            sampler: The sampler used to sample fantasy observations. Optional
                if `num_fantasies` is specified.
            objective: The objective under which the samples are evaluated. If
                `None` or a ScalarizedObjective, then the analytic posterior mean
                is used, otherwise the objective is MC-evaluated (using
                inner_sampler).
            inner_sampler: The sampler used for inner sampling. Ignored if the
                objective is `None` or a ScalarizedObjective.
            X_pending: A `m x d`-dim Tensor of `m` design points that have
                points that have been submitted for function evaluation
                but have not yet been evaluated.
            current_value: The current value, i.e. the expected best objective
                given the observed points `D`. If omitted, forward will not
                return the actual KG value, but the expected best objective
                given the data set `D u X`.
        """
        if sampler is None:
            if num_fantasies is None:
                raise ValueError(
                    "Must specify `num_fantasies` if no `sampler` is provided."
                )
            # base samples should be fixed for joint optimization over X, X_fantasies
            sampler = SobolQMCNormalSampler(
                num_samples=num_fantasies, resample=False, collapse_batch_dims=True
            )
        elif num_fantasies is not None:
            if sampler.sample_shape != torch.Size([num_fantasies]):
                raise ValueError(
                    f"The sampler shape must match num_fantasies={num_fantasies}."
                )
        else:
            num_fantasies = sampler.sample_shape[0]
        super(MCAcquisitionFunction, self).__init__(model=model)
        # if not explicitly specified, we use the posterior mean for linear objs
        if isinstance(objective, MCAcquisitionObjective) and inner_sampler is None:
            inner_sampler = SobolQMCNormalSampler(
                num_samples=128, resample=False, collapse_batch_dims=True
            )
        if objective is None and model.num_outputs != 1:
            raise UnsupportedError(
                "Must specify an objective when using a multi-output model."
            )
        self.sampler = sampler
        self.objective = objective
        self.set_X_pending(X_pending)
        self.inner_sampler = inner_sampler
        self.num_fantasies = num_fantasies
        self.current_value = current_value

    @t_batch_mode_transform()
    def forward(self, X):
        r"""Evaluate qKnowledgeGradient on the candidate set `X`.

        Args:
            X: A `b x (q + num_fantasies) x d` Tensor with `b` t-batches of
                `q + num_fantasies` design points each. We split this X tensor
                into two parts in the `q` dimension (`dim=-2`). The first `q`
                are the q-batch of design points and the last num_fantasies are
                the current solutions of the inner optimization problem.

                `X_fantasies = X[..., -num_fantasies:, :]`
                `X_fantasies.shape = b x num_fantasies x d`

                `X_actual = X[..., :-num_fantasies, :]`
                `X_actual.shape = b x q x d`

        Returns:
            A Tensor of shape `b`. For t-batch b, the q-KG value of the design
                `X_actual[b]` is averaged across the fantasy models, where
                `X_fantasies[b, i]` is chosen as the final selection for the
                `i`-th fantasy model.
                NOTE: If `current_value` is not provided, then this is not the
                true KG value of `X_actual[b]`, and `X_fantasies[b, : ]` must be
                maximized at fixed `X_actual[b]`.
        """
#         print('forward', X.shape) #torch.Size([5, 1, 7])
        X_actual, X_fantasies = _split_fantasy_points(X=X, n_f=self.num_fantasies)

        # We only concatenate X_pending into the X part after splitting
        if self.X_pending is not None:
            X_actual = torch.cat(
                [X_actual, match_batch_shape(self.X_pending, X_actual)], dim=-2
            )

        # construct the fantasy model of shape `num_fantasies x b`
        fantasy_model = self.model.fantasize(
            X=X_actual, sampler=self.sampler, observation_noise=True
        )

        # get the value function
        value_function = _get_value_function(
            model=fantasy_model, objective=self.objective, sampler=self.inner_sampler
        )

        # make sure to propagate gradients to the fantasy model train inputs
        with botorch.settings.propagate_grads(True):
            values = value_function(X=X_fantasies)  # num_fantasies x b

        if self.current_value is not None:
            values = values - self.current_value

        # return average over the fantasy samples
        return values.mean(dim=0)

    def get_augmented_q_batch_size(self, q):
        r"""Get augmented q batch size for one-shot optimzation.

        Args:
            q: The number of candidates to consider jointly.

        Returns:
            The augmented size for one-shot optimzation (including variables
            parameterizing the fantasy solutions).
        """
        return q + self.num_fantasies

    def extract_candidates(self, X_full):
        r"""We only return X as the set of candidates post-optimization.

        Args:
            X_full: A `b x (q + num_fantasies) x d`-dim Tensor with `b`
                t-batches of `q + num_fantasies` design points each.

        Returns:
            A `b x q x d`-dim Tensor with `b` t-batches of `q` design points each.
        """
        return X_full[..., : -self.num_fantasies, :]

In [14]:
def pdict_to_parr(d, bound_dict=True):

    list(param_bounds['beta'].keys()) + ['mu']
    arr = torch.stack([
        torch.tensor(d['beta']['household']),
        torch.tensor(d['beta']['education']),
        torch.tensor(d['beta']['office']),
        torch.tensor(d['beta']['social']),
        torch.tensor(d['beta']['supermarket']),
        torch.tensor(d['beta']['transport']),
        torch.tensor(d['mu']),
    ])
    return arr

def parr_to_pdict(arr, bound_dict=True):
    
    d = {
        'beta': {
            'household': arr[0],
            'education': arr[1],
            'office': arr[2],
            'social': arr[3],
            'supermarket': arr[4],
            'transport': arr[5],
        },
        'mu': arr[6]
    }
    return d

example = {
    'beta': {
        'household': torch.tensor([3.]),
        'education': torch.tensor([0.5]),
        'office': torch.tensor([10.]),
        'social': torch.tensor([7.]),
        'supermarket': torch.tensor([4.]),
        'transport': torch.tensor([2.])
    },
      'mu': torch.tensor([0.5000])}

assert(example == parr_to_pdict(pdict_to_parr(example)))


In [15]:
N_DAYS, N_AGE = new_cases.shape
G_obs = torch.tensor(new_cases).reshape(N_DAYS * N_AGE) # flattened
N_PARAMS = 7

SIM_BOUNDS = pdict_to_parr(param_bounds)
BO_BOUNDS = torch.stack([torch.zeros(N_PARAMS), torch.ones(N_PARAMS)]).T

print('Simulation bounds:')
pprint.pprint(parr_to_pdict(SIM_BOUNDS))
print('\nBO bounds (converted):')
pprint.pprint(parr_to_pdict(BO_BOUNDS))


Simulation bounds:
{'beta': {'education': tensor([ 0., 10.]),
          'household': tensor([ 0., 10.]),
          'office': tensor([ 0., 10.]),
          'social': tensor([ 0., 10.]),
          'supermarket': tensor([ 0., 10.]),
          'transport': tensor([ 0., 10.])},
 'mu': tensor([0., 1.])}

BO bounds (converted):
{'beta': {'education': tensor([0., 1.]),
          'household': tensor([0., 1.]),
          'office': tensor([0., 1.]),
          'social': tensor([0., 1.]),
          'supermarket': tensor([0., 1.]),
          'transport': tensor([0., 1.])},
 'mu': tensor([0., 1.])}


In [16]:
# simulation
def composite_simulation(norm_params):
        
    # un-normalize normalized params to obtain simulation parameters
    params = transforms.unnormalize(norm_params, SIM_BOUNDS.T)
 
    # run simulation
    
    # get results
    out = params.sum() * torch.ones((N_DAYS, N_AGE))
    
    # flatten
    means = out.reshape(1, N_DAYS * N_AGE)  
    
    # standard error of mean
    SEM = 0.1 * torch.rand_like(means)
    
    return means, SEM

# objective
def composite_squared_loss(G):
    
#     print('composite_squared_loss', G.shape, G_obs.shape)
    
#     return - (G - G_obs).pow(2).sum(dim=-1)
    return - (G).pow(2).sum(dim=-1)
   

composite_squared_objective = GenericMCObjective(composite_squared_loss)

# select objective
objective = composite_squared_objective

In [17]:
# Model initialization
def standardize(Y, only_stddev) :
    r"""Standardizes (zero mean, unit variance) a n x m tensor by dim=0.
    Returns standardized 'y' and std.dev. of 'y'
    """
    stddim = -1 if Y.dim() < 2 else -2
    Y_std = Y.std(dim=stddim, keepdim=True)
    Y_std = Y_std.where(Y_std >= 1e-9, torch.full_like(Y_std, 1.0))
    if only_stddev:
        return Y_std
    else:
        return (Y - Y.mean(dim=stddim, keepdim=True)) / Y_std


def generate_initial_data(n):
    if n <= 0:
        raise ValueError('qKnowledgeGradient and GP needs at least one observation to be defined properly.')
    
    # sobol sequence
    # new_thetas: [n, N_PARAMS]
    new_thetas = torch.tensor(sobol_seq.i4_sobol_generate(N_PARAMS, n), dtype=torch.float)
    
    # new_G, new_G_sem: [n, N_DAYS * N_AGE]
    new_G = torch.zeros((n, N_DAYS * N_AGE), dtype=torch.float)
    new_G_sem = torch.zeros((n, N_DAYS * N_AGE), dtype=torch.float)
    for i in range(n):
        print(f'Simulating inital setting {i+1}/{n} ... ', end='')
        # mean and standard error of mean (sem) of every simulation output
        G, G_sem = composite_simulation(new_thetas[i, :])
        new_G[i, :] = G
        new_G_sem[i, :] = G_sem
        print('done.')
            
    # compute best objective from simulations
    best_f = objective(new_G).max().item()
    
    return new_thetas, new_G, new_G_sem, best_f
    
def initialize_model(train_x, train_y, train_y_sem):

    # standardize (zero mean, unit variance)
    # 1) get standard deviation (i.e. multiplier when standardized)
    Y_stddev = standardize(train_y, only_stddev=True)
    
    # 2_ standardize y to make GP hyperparameter tuning work out
    train_y = standardize(train_y, only_stddev=False)
    
    # 3) scale standard error of mean `train_y_sem` by the same amount as train_y in `standardize`
    #    since Var[aX] = a^2 Var[X], we have SEM(aX) = a SEM(X)
    train_y_sem /= Y_stddev
    train_ynoise = train_y_sem.pow(2.0) # noise is in variance units
    
    # train 
    model = FixedNoiseGP(train_x, train_y, train_ynoise)
    
    # "Loss" for GPs - the marginal log likelihood
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
        
    return mll, model



In [18]:
def optimize_acqf_and_get_observation(acq_func, num_restarts, raw_samples, batch_limit, maxiter):
    """
    Optimizes the acquisition function, and returns a new candidate and a noisy observation.
    botorch defaults:  num_restarts=10, raw_samples=256, batch_limit=5, maxiter=200
    """
    
    batch_initial_conditions = gen_one_shot_kg_initial_conditions(
        acq_function=acq_func,
        bounds=BO_BOUNDS.T,
        q=1,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
        options={"batch_limit": batch_limit, "maxiter": maxiter},
    )
    
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=BO_BOUNDS.T,
        q=1,
        num_restarts=num_restarts,
        raw_samples=raw_samples,  # used for intialization heuristic
        options={"batch_limit": batch_limit, "maxiter": maxiter},
        batch_initial_conditions=batch_initial_conditions
    )
        
    # proposed evaluation 
    new_theta = candidates.detach()
    
    # observe new noisy function evaluation
    new_G, new_G_sem = composite_simulation(new_theta)

    return new_theta, new_G, new_G_sem

In [ ]:
INIT_SAMPLES = 5 # initial random evaluation points
N_BATCH = 100 # rounds of BO after the initial random batch
MC_SAMPLES = 256 # samples for MC acquisition function 
verbose = True

# Knowledge gradient
num_fantasies = 64
num_restarts = 10
raw_samples = 256
batch_limit = 5
maxiter = 20

best_observed = []

# generate initial training data and initialize model
train_theta, train_G, train_G_sem, best_observed_y = generate_initial_data(n=INIT_SAMPLES)
mll, model = initialize_model(train_theta, train_G, train_G_sem)

print('Best objective (init phase): ', best_observed_y)

best_observed.append(best_observed_y)

# run N_BATCH rounds of BayesOpt after the initial random batch
for tt in range(1, N_BATCH + 1):
    
    t0 = time.time()

    # fit the GP model
    fit_gpytorch_model(mll)
    
    # acquisition function
    acqf = qKnowledgeGradient(
        model=model,
        objective=objective,
        num_fantasies=num_fantasies,
    )
    
    # optimize and get new observation
    new_theta, new_G, new_G_sem = optimize_acqf_and_get_observation(
        acqf, num_restarts=num_restarts, raw_samples=raw_samples, 
        batch_limit=batch_limit, maxiter=maxiter)
    
    train_theta = torch.cat([train_theta, new_theta], dim=0) # [tt, N_PARAMS]
    train_G = torch.cat([train_G, new_G], dim=0) # [tt, N_DAYS * N_AGE]
    train_G_sem = torch.cat([train_G_sem, new_G_sem], dim=0) # [tt, N_DAYS * N_AGE]
    
    # update progress
    best_observed_y = objective(train_G).max().item()
    best_observed.append(best_observed_y)
    
    # re-initialize the models so they are ready for fitting on next iteration
    mll, model = initialize_model(
        train_theta, 
        train_G, 
        train_G_sem,
    )

    t1 = time.time()
    

    if verbose:
        print(
            f"{tt:>2}: best = "
            f"{best_observed_y:>4.2f}, "
            f"{objective(new_G).item():>4.2f}, "
            f"{new_theta.detach()}, "
            f"time = {t1 - t0:>4.2f}."
            , end="\n"
        )
    else:
        print(".", end="")


Simulating inital setting 1/5 ... done.
Simulating inital setting 2/5 ... done.
Simulating inital setting 3/5 ... done.
Simulating inital setting 4/5 ... done.
Simulating inital setting 5/5 ... done.
Best objective (init phase):  -93336.375
 1: best = -93336.38, -143454.34, tensor([[0.1878, 0.3637, 0.7734, 0.7658, 0.9368, 0.6866, 0.3608]]), time = 29.06.
 2: best = -93336.38, -139006.23, tensor([[0.3768, 0.2303, 1.0000, 0.8424, 0.8443, 0.3661, 0.3186]]), time = 26.03.
 3: best = -93336.38, -130109.94, tensor([[0.2066, 0.9520, 0.4193, 0.8840, 0.3722, 0.6384, 0.9888]]), time = 35.19.
 4: best = -93336.38, -100325.67, tensor([[0.6246, 0.7884, 0.3191, 0.2098, 0.9623, 0.2003, 0.3170]]), time = 38.67.
 5: best = -53942.82, -53942.82, tensor([[0.1807, 0.4933, 0.8484, 0.4128, 0.0192, 0.2649, 0.8029]]), time = 36.17.
 6: best = -53942.82, -145032.45, tensor([[0.8783, 0.1140, 0.4989, 0.9240, 0.5166, 0.8363, 0.0270]]), time = 35.36.
 7: best = -53942.82, -137149.12, tensor([[0.5455, 0.4948, 0.345